In [1]:
import pandas as pd
import mysql.connector 
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('business_id.csv', 'business_id')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Create table if it doesn't exist
        columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

Processing business_id.csv
NaN values before replacement:
business_id        0
name               0
address          324
city               0
state              0
postal_code        4
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       441
categories         0
hours           3364
dtype: int64

✅ Finished inserting data into `business_id`



In [2]:
import pandas as pd
import mysql.connector 
import os

csv_files = [
    ('checkin.csv', 'checkin')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(col, dtype):
    # Special handling for checkin_date column (can contain very long strings)
    if col.lower() in ["date", "checkin_date"]:
        return "LONGTEXT"
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "LONGTEXT"  # default to LONGTEXT for safety

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Rename BEFORE creating schema
        if 'date' in df.columns:
            df.rename(columns={'date': 'checkin_date'}, inplace=True)

        # ✅ Now MySQL table & insert will use "checkin_date"
        columns = ', '.join([f'`{col}` {get_sql_type(col, df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # drop old wrong table
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

Processing checkin.csv
NaN values before replacement:
Column1    0
Column2    0
dtype: int64

✅ Finished inserting data into `checkin`



In [1]:
import pandas as pd
import mysql.connector 
import os

csv_files = [
    ('tip.csv', 'tip')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(col, dtype):
    # Special handling for checkin_date column (can contain very long strings)
    if col.lower() in ["date", "checkin_date"]:
        return "LONGTEXT"
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "LONGTEXT"  # default to LONGTEXT for safety

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Rename BEFORE creating schema
        if 'date' in df.columns:
            df.rename(columns={'date': 'checkin_date'}, inplace=True)

        # ✅ Now MySQL table & insert will use "checkin_date"
        columns = ', '.join([f'`{col}` {get_sql_type(col, df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # drop old wrong table
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

Processing tip.csv
NaN values before replacement:
user_id             0
business_id         0
text                7
date                0
compliment_count    0
dtype: int64

✅ Finished inserting data into `tip`



In [2]:
import pandas as pd
import mysql.connector 
import os

csv_files = [
    ('new_checkin.csv', 'new_checkin')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(col, dtype):
    # Special handling for checkin_date column (can contain very long strings)
    if col.lower() in ["date", "checkin_date"]:
        return "LONGTEXT"
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "LONGTEXT"  # default to LONGTEXT for safety

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Rename BEFORE creating schema
        if 'date' in df.columns:
            df.rename(columns={'date': 'checkin_date'}, inplace=True)

        # ✅ Now MySQL table & insert will use "checkin_date"
        columns = ', '.join([f'`{col}` {get_sql_type(col, df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # drop old wrong table
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

Processing new_checkin.csv
NaN values before replacement:
business_id    0
date           0
dtype: int64

✅ Finished inserting data into `new_checkin`



In [3]:
import pandas as pd
import mysql.connector 
import os

csv_files = [
    ('reviews.csv', 'reviews')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(col, dtype):
    # Special handling for checkin_date column (can contain very long strings)
    if col.lower() in ["date", "checkin_date"]:
        return "LONGTEXT"
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "LONGTEXT"  # default to LONGTEXT for safety

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Rename BEFORE creating schema
        if 'date' in df.columns:
            df.rename(columns={'date': 'checkin_date'}, inplace=True)

        # ✅ Now MySQL table & insert will use "checkin_date"
        columns = ', '.join([f'`{col}` {get_sql_type(col, df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # drop old wrong table
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

Processing reviews.csv
NaN values before replacement:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

✅ Finished inserting data into `reviews`



In [1]:
import pandas as pd
import mysql.connector 
import os

csv_files = [
    ('user.csv', 'user')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='vansh@11singH',
    database='demo2_msql_english'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/hp/Desktop/screenshot'

# Convert pandas dtype to MySQL column type
def get_sql_type(col, dtype):
    # Special handling for checkin_date column (can contain very long strings)
    if col.lower() in ["date", "checkin_date"]:
        return "LONGTEXT"
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_bool_dtype(dtype):
        return "BOOLEAN"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME"
    else:
        return "LONGTEXT"  # default to LONGTEXT for safety

# Process each CSV file
for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        df = pd.read_csv(file_path)
        df = df.where(pd.notnull(df), None)

        print(f"Processing {csv_file}")
        print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Rename BEFORE creating schema
        if 'date' in df.columns:
            df.rename(columns={'date': 'checkin_date'}, inplace=True)

        # ✅ Now MySQL table & insert will use "checkin_date"
        columns = ', '.join([f'`{col}` {get_sql_type(col, df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
        cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")  # drop old wrong table
        cursor.execute(create_table_query)

        # Prepare insert query
        columns_str = ', '.join([f'`{col}`' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

        # Convert dataframe rows to list of tuples
        values = [tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False, name=None)]

        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            cursor.executemany(insert_query, batch)

        conn.commit()
        print(f"✅ Finished inserting data into `{table_name}`\n")

    except Exception as e:
        print(f"❌ Error processing {csv_file}: {e}\n")

# Close connection
cursor.close()
conn.close()

C:\Users\hp\AppData\Local\Temp\ipykernel_10684\1818286120.py:42: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processing user.csv
NaN values before replacement:
user_id                     0
name                       37
review_count                0
yelping_since               0
useful                      0
funny                       0
cool                        0
elite                 1896699
friends                878551
fans                        0
average_stars               0
compliment_hot              0
compliment_more             0
compliment_profile          0
compliment_cute             0
compliment_list             0
compliment_note             0
compliment_plain            0
compliment_cool             0
compliment_funny            0
compliment_writer           0
compliment_photos           0
dtype: int64

✅ Finished inserting data into `user`

